In [1]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
from torchvision import transforms

import os
import zipfile

In [5]:
if torch.cuda.is_available():
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev) 

In [8]:
base_dir = "../input/cian-datafest-2019"
train_dir = os.path.join(base_dir, "train.zip")
test_dir = os.path.join(base_dir, "test.zip")

with zipfile.ZipFile(train_dir,"r") as z:
    z.extractall()
    
with zipfile.ZipFile(test_dir,"r") as z:
    z.extractall()

In [9]:
transform = transforms.Compose([transforms.Resize((56,56)), transforms.ToTensor()])

train_data = torchvision.datasets.ImageFolder('./train', 
                                              transform=transform)

trainset = torch.utils.data.Subset(train_data, list(range(0, len(train_data), 6)))

train_set, val_set = torch.utils.data.random_split(trainset, [6000, 3221])
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=500, 
                                                shuffle=True, num_workers=2)
val_data_loader = torch.utils.data.DataLoader(val_set, batch_size=500, 
                                              shuffle=True, num_workers=2)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, padding=1, stride=1)
        self.pooling_layer1 = nn.MaxPool2d(kernel_size=2)
        self.conv_layer2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=4)
        self.pooling_layer2 = nn.AvgPool2d(kernel_size=2)
        self.conv_layer3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=2, padding=1, stride=2)
        
        self.linear_layer1 = nn.Linear(in_features=1568, out_features=128)
        self.linear_layer2 = nn.Linear(in_features=128, out_features=100)
        self.linear_layer3 = nn.Linear(in_features=100, out_features=1)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        output_1 = self.relu(self.conv_layer1(x))
        output_2 = self.pooling_layer1(output_1)
        output_3 = self.relu(self.conv_layer2(output_2))
        output_4 = self.pooling_layer2(output_3)
        output_5 = self.relu(self.conv_layer3(output_4))
        
        output_6 = torch.flatten(output_5, 1)
        
        output_7 = self.relu(self.linear_layer1(output_6))
        output_8 = self.relu(self.linear_layer2(output_7))
        output_9 = self.linear_layer3(output_8)
        
        return output_9

In [ ]:
model = CNN().to(device)
print(model)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# обучение
torch.manual_seed(10)

epochs = 5

total_step = len(train_data_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
               .format(epoch+1, epochs, i+1, total_step, loss.item()))

In [ ]:
# оценка качества
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_data_loader:
        images = images.to(device)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)
        
        outputs = model(images)
        predicted = (torch.sigmoid(outputs.data) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

**Моя архитектура**

In [10]:
class my_CNN(nn.Module):
    def __init__(self):
        super(my_CNN, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=2)
        self.conv_layer3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2, padding=1)
        self.pooling_layer1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv_layer5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=2, padding=1)
        self.pooling_layer2 = nn.MaxPool2d(kernel_size=2)
        
        self.conv_layer6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=2, padding=2, stride=2)
        self.conv_layer7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3)
        self.conv_layer8 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2)
        self.pooling_layer3 = nn.AvgPool2d(kernel_size=2)
        
        self.linear_layer1 = nn.Linear(in_features=2304, out_features=1024)
        self.dropout_1 = nn.Dropout(p=0.5)
        self.linear_layer2 = nn.Linear(in_features=1024, out_features=512)
        self.dropout_2 = nn.Dropout(p=0.3)
        self.linear_layer3 = nn.Linear(in_features=512, out_features=128)
        self.dropout_3 = nn.Dropout(p=0.15)
        self.linear_layer4 = nn.Linear(in_features=128, out_features=1)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        output = self.relu(self.conv_layer1(x))
        output = self.relu(self.conv_layer2(output))
        output = self.relu(self.conv_layer3(output))
        output = self.pooling_layer1(output)
        
        output = self.relu(self.conv_layer4(output))
        output = self.relu(self.conv_layer5(output))
        output = self.pooling_layer2(output)
        
        output = self.relu(self.conv_layer6(output))
        output = self.relu(self.conv_layer7(output))
        output = self.relu(self.conv_layer8(output))
        output = self.pooling_layer3(output)
        
        output = torch.flatten(output, 1)
        
        output = self.linear_layer1(output)
        output = self.dropout_1(output)
        output = self.linear_layer2(output)
        output = self.dropout_2(output)
        output = self.linear_layer3(output)
        output = self.dropout_3(output)
        output = self.linear_layer4(output)
        
        return output

In [11]:
model = my_CNN().to(device)
print(model)

In [13]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
# обучение
torch.manual_seed(10)

epochs = 5

total_step = len(train_data_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
               .format(epoch+1, epochs, i+1, total_step, loss.item()))

In [15]:
# оценка качества
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_data_loader:
        images = images.to(device)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)
        
        outputs = model(images)
        predicted = (torch.sigmoid(outputs.data) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))